In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os
import numpy as np
np.random.seed(777)
import math
import keras
import keras.backend as K
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score
from keras.regularizers import l2
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications import DenseNet201
from keras.applications import DenseNet121

from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import SeparableConv2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

In [ ]:
train_dir = '/content/drive/My Drive/DDSM_CBIS_Enhanced/train/'
test_dir = '/content/drive/My Drive/DDSM_CBIS_Enhanced/test/'
valid_dir = '/content/drive/My Drive/DDSM_CBIS_Enhanced/valid/'

extracted_features_dir = '/content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/'
model_name = "Rahman_Xcep"

In [ ]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)

Keras Version 2.4.3
tensorflow Version 2.3.0


In [ ]:
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 1000

In [ ]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(valid_dir):
    print (root, len(files))

/content/drive/My Drive/DDSM_CBIS_Enhanced/train/ 0
/content/drive/My Drive/DDSM_CBIS_Enhanced/train/benign 572
/content/drive/My Drive/DDSM_CBIS_Enhanced/train/malignant 522
******************************
/content/drive/My Drive/DDSM_CBIS_Enhanced/test/ 0
/content/drive/My Drive/DDSM_CBIS_Enhanced/test/malignant 147
/content/drive/My Drive/DDSM_CBIS_Enhanced/test/benign 230
******************************
/content/drive/My Drive/DDSM_CBIS_Enhanced/valid/ 0
/content/drive/My Drive/DDSM_CBIS_Enhanced/valid/benign 101
/content/drive/My Drive/DDSM_CBIS_Enhanced/valid/malignant 100


In [ ]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 821 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 377 images belonging to 2 classes.


In [ ]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

nb_test_samples = len(test_generator.filenames)
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

# nb_test_without_aug_samples = len(test_generator_without_aug.filenames)
# predict_size_test_without_aug = int(math.ceil(nb_test_without_aug_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
# print("nb_test_without_aug_samples:", nb_test_without_aug_samples)
# print("predict_size_test_without_aug_samples:", predict_size_test_without_aug)
print("nb_test_samples:", nb_test_samples)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 821
nb_validation_samples: 50

predict_size_train: 26
predict_size_validation: 2
nb_test_samples: 377
predict_size_test: 12

 num_classes: 2


In [ ]:
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet169
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet


input_tensor = Input(shape = input_shape)  
model = Xception(weights='imagenet', include_top=False, pooling = 'avg', input_tensor=input_tensor)
bottleneck_final_model = model

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [ ]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [ ]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [ ]:
# example of saving sub-models for later use in a stacking ensemble
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import os , shutil
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min')
save = ModelCheckpoint('/content/drive/My Drive/Brain-Tumor-Detection-master/extracted/my_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=50, verbose=1, epsilon=1e-3, mode='min')

 
# fit model on dataset
def fit_model(trainX,trainy):
  dropout_rate = 0.4
  model = Sequential()
  # model.add(Flatten(input_shape=train_data.shape[1:]))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(num_classes, activation=tf.nn.softmax))
  adam_opt2=Adam(lr = 0.0001, beta_1=0.6, beta_2=0.9, amsgrad=False)
  model.compile(optimizer=adam_opt2, loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(trainX,trainy,epochs=500,validation_data=(validation_data, validation_labels),batch_size=batch_size,verbose=0,callbacks=[stop, save, reduce_lr])
  return model



trainX = train_data
testX  = test_data
trainy = train_generator.classes
trainy = to_categorical(trainy, num_classes=2)
testy  = test_generator.classes
testy_enc = to_categorical(testy, num_classes=2)


print(trainX.shape, trainy.shape)
print(testX.shape, testy.shape)
# create directory for models
base_dir = '/content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/'
#os.mkdir(base_dir)
# fit and save models
n_members = 5
for i in range(n_members):
	# fit model
	model = fit_model(trainX, trainy)
	# save model
	filename = '/content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_' + str(i + 1) + '.h5'
	model.save(filename)
	print('>Saved %s' % filename)

(821, 2048) (821, 2)
(377, 2048) (377,)

Epoch 00073: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 00073: early stopping
>Saved /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_1.h5

Epoch 00062: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 00062: early stopping
>Saved /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_2.h5

Epoch 00061: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 00073: early stopping
>Saved /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_3.h5

Epoch 00067: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 00067: early stopping
>Saved /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_4.h5

Epoch 00067: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 00067: early stopping
>Saved /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_5.h5


In [ ]:
# load models from file
from keras.models import load_model
from numpy import dstack
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack




def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = '/content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [ ]:
# load all models
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

>loaded /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_1.h5
>loaded /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_2.h5
>loaded /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_3.h5
>loaded /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_4.h5
>loaded /content/drive/My Drive/DDSM_CBIS_Enhanced/extracted_features/model_5.h5
Loaded 5 models


In [ ]:
# evaluate standalone models on test dataset
for model in members:
	testy_enc = to_categorical(testy)
	_, acc = model.evaluate(testX, testy_enc, verbose=0)
	print('Model Accuracy: %.3f' % acc)

Model Accuracy: 0.692
Model Accuracy: 0.719
Model Accuracy: 0.692
Model Accuracy: 0.687
Model Accuracy: 0.698


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
for model in members:
  preds = model.predict(test_data)

  predictions = [i.argmax() for i in preds]
  y_true = [i.argmax() for i in test_labels]
  cm = confusion_matrix(y_pred=predictions, y_true=y_true)

  print('Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))
  plt.rcParams["axes.grid"] = False
  plt.rcParams.update({'font.size': 20})

  labels = []

  label = test_generator.class_indices
  indexlabel = dict((value, key) for key, value in label.items())

  for k,v in indexlabel.items():
      labels.append(v)

  from sklearn.metrics import confusion_matrix
  import itertools
  def plot_confusion_matrix(cm, classes,
                            normalize=False,
                            title='Confusion matrix',
                            cmap=plt.cm.Blues):
      accuracy = np.trace(cm) / float(np.sum(cm))
      misclass = 1 - accuracy
      if normalize:
          cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
          print("Normalized confusion matrix")
      else:
          print('Confusion Matrix')

      print(cm)
  #     fig = plt.figure()
      plt.imshow(cm, interpolation='nearest', cmap=cmap)
  #     plt.title(title)
  #     plt.colorbar()
      tick_marks = np.arange(len(classes))
      plt.xticks(tick_marks, classes, rotation=45)
      plt.yticks(tick_marks, classes)

      fmt = '.2f' if normalize else 'd'
      thresh = cm.max() / 2.
      for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
          plt.text(j, i, format(cm[i, j], fmt),
                   horizontalalignment="center",
                   color="white" if cm[i, j] > thresh else "black")

      plt.tight_layout()
      plt.ylabel('True label')
      plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
      #plt.savefig('/content/drive/My Drive/data_v3/extracted_features/confusion_marix.png', bbox_inches='tight', dpi = 100) 


  #plt.figure(figsize=(10,10))
  #plot_confusion_matrix(cm, classes=labels, title=' ')

  from sklearn.preprocessing import LabelEncoder
  from sklearn.metrics import confusion_matrix, classification_report
  y_pred=predictions
  y_pred_probabilities=y_pred

  # y_pred = np.argmax(y_pred,axis = 1) 
  y_actual = y_true

  classnames=[]
  for classname in test_generator.class_indices:
      classnames.append(classname)

  confusion_mtx = confusion_matrix(y_actual, y_pred) 
  print(confusion_mtx)
  target_names = classnames
  print(classification_report(y_actual, y_pred, target_names=target_names))



  from keras.preprocessing.image import ImageDataGenerator, load_img
  fnames = test_generator.filenames
  ground_truth = test_generator.classes

  label2index = test_generator.class_indices
  idx2label = dict((v,k) for k,v in label2index.items())

  predictions = model.predict(test_data)
  predicted_classes = np.argmax(predictions,axis=1)

  errors = np.where(predicted_classes != ground_truth)[0]
  no_error = len(errors)
  print("No of errors = {}/{}".format(len(errors),test_generator.samples))
  #no_errors.append(no_error)

  # Show the errors
  for i in range(len(errors)):
      pred_class = np.argmax(predictions[errors[i]])
      pred_label = idx2label[pred_class]

      title = 'filename:{},Original label:{}, Prediction :{}, confidence : {:.3f}, class ID : {}'.format(
              fnames[errors[i]],
              fnames[errors[i]].split('/')[0],
              pred_label,
              predictions[errors[i]][pred_class], pred_class)

      original = load_img('{}/{}'.format(test_dir,fnames[errors[i]]))
      #plt.figure(figsize=[7,7])
      #plt.axis('off')
      #plt.title(title)
      #plt.imshow(original)
      print(title)
  #plt.show()


Accuracy 0.6923076923076923
[[171  59]
 [ 57  90]]
              precision    recall  f1-score   support

      benign       0.75      0.74      0.75       230
   malignant       0.60      0.61      0.61       147

    accuracy                           0.69       377
   macro avg       0.68      0.68      0.68       377
weighted avg       0.69      0.69      0.69       377

No of errors = 116/377
filename:benign/Mass-Test_P_00032_RIGHT_CC_ROI_Mass_B_1.png .png,Original label:benign, Prediction :malignant, confidence : 0.806, class ID : 1
filename:benign/Mass-Test_P_00126_RIGHT_CC_ROI_Mass_B_1.png .png,Original label:benign, Prediction :malignant, confidence : 0.586, class ID : 1
filename:benign/Mass-Test_P_00126_RIGHT_MLO_ROI_Mass_B_1.png .png,Original label:benign, Prediction :malignant, confidence : 0.507, class ID : 1
filename:benign/Mass-Test_P_00131_LEFT_CC_ROI_Mass_B_1.png .png,Original label:benign, Prediction :malignant, confidence : 0.525, class ID : 1
filename:benign/Mass-Te

In [ ]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [ ]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = LogisticRegression()
	model.fit(stackedX, inputy)
	return model

In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, testX, testy)

In [ ]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [ ]:
# evaluate model on test set as stacked ensemble

yhat = stacked_prediction(members, model, testX)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.762
